## Transfer Learning
Use pre-trained backbone from `run_baseline_pretrain_moco.ipynb`

### Environment Setup

In [1]:
# import libraries
import os
import sys
import argparse
from tqdm import tqdm

import numpy as np
import seaborn as sns
import pandas as pd
import math
from copy import deepcopy

%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import display

import pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Collab Needs

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define filepaths to required input (scripts, data) and outputs

In [3]:
# ----------------------------------------------------
# Inputs root
# ----------------------------------------------------
FP_ROOT="/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/"

# ----------------------------------------------------
# Dataset info
# ----------------------------------------------------
# Images zip files

# Chexpert
DATASET_PATH_SPLIT = FP_ROOT + "Data/CheXpert_reduced_dataset_split_multiclass.zip"

# Labels
# ChexPert - MultiClass
TRAIN_LABELS_CSV = FP_ROOT + "Data/3_final_project_updated_names_train_transfer.csv"
VAL_LABELS_CSV = FP_ROOT + "Data/3_final_project_updated_names_val_transfer.csv"
TEST_LABELS_CSV = FP_ROOT + "Data/3_final_project_updated_names_test_transfer.csv"

# ----------------------------------------------------
# Model SRC
# ----------------------------------------------------
SRC_ROOT = FP_ROOT + "src/"

# ----------------------------------------------------
# Outputs
# ----------------------------------------------------
ROOT_ARTIFACT_SAVE = FP_ROOT + "artifacts/"

In [4]:
# Add the project /src to the system path
if SRC_ROOT not in sys.path:
    sys.path.append(SRC_ROOT)
    print(f"Added {SRC_ROOT} to sys.path")

print(sys.path)

Added /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src/ to sys.path
['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/tmp/tmpxttkeko3', '/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src/']


## Unzip data

In [5]:
# Unzip the dataset (image) files to /tmp
DATA_DEST_UNZIPPED = "/tmp/CheXpert_dataset/"
os.makedirs(DATA_DEST_UNZIPPED, exist_ok=True)
!unzip "{DATASET_PATH_SPLIT}" -d {DATA_DEST_UNZIPPED} # need to use "" to accomdate space

Streaming output truncated to the last 5000 lines.
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48391_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48398_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48399_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48424_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48425_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48461_study1_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patient48471_study3_view1_frontal.jpg  
  inflating: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train/patie

In [5]:
# Update for the unzipped sub-name
DATA_DEST_UNZIPPED = "/tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/"

## 2) Fine tune the classifier

In [6]:
%cd "/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src"

/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/src


In [7]:
# Load in the pre-trained baseline MoCo model

# Extract the checkpoint state dict
MOCO_BACKBONE_SAVE_PATH = f"{ROOT_ARTIFACT_SAVE}/11_19_Moco_Complete_Resnet50/moco_resnet50_encoder.pth"

### Run finetuning using the finetune_resnet module

In [9]:
ROOT_ARTIFACT_SAVE

'/content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts/'

## CheXpert
MultiClass - One Label Per data row

In [8]:
! python finetune_resnet.py \
    --train_csv "$TRAIN_LABELS_CSV" \
    --val_csv "$VAL_LABELS_CSV" \
    --test_csv "$TEST_LABELS_CSV" \
    --root_dir "$DATA_DEST_UNZIPPED" \
    --pretrained_encoder "$MOCO_BACKBONE_SAVE_PATH" \
    --artifact_root "$ROOT_ARTIFACT_SAVE" \
    --n_epochs 20 \
    --batch_size 64 \
    --num_classes 14 \
    --subtitle "CheXpert_Multiclass_resnet50" \
    --label_col "Label"

Created log file:  /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts/training_log_20251120_022134.txt
Loading train dataset...
CSV: /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/3_final_project_updated_names_train_transfer.csv
 * Images - Train Root Directory: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/train
Unique labels in column 'Label': [ 0  3 12  8  9  5  2  6 13 10  7  1  4 11]
Loading val dataset...
CSV: /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/Data/3_final_project_updated_names_val_transfer.csv
 * Images - Val Root Directory: /tmp/CheXpert_dataset/CheXpert_reduced_dataset_split_multiclass/val
Unique labels in column 'Label': [ 5  2  3  0  9  7 12  8 10  6 13  4  1 11]
[2025-11-20 02:21:34] Loading pretrained encoder from: /content/drive/MyDrive/Colab Notebooks/7_Py_DL/FP/artifacts//11_19_Moco_Complete_Resnet50/moco_resnet50_encoder.pth
[2025-11-20 02:21:34] Detected 'encoder_q_state' in checkpoint.
[2025-11-20 02:21:34] Loaded pr